In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import random
import re
import statistics as st
%matplotlib inline
# importing MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(pd.__version__)
print(np.__version__)


1.5.3
1.23.5


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stp = stopwords.words('english')
print(stp)
print(len(stp))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
path = '/content/drive/MyDrive/Omar/AAAI-Submission/'
train = pd.read_csv(path + 'train.csv')
val = pd.read_csv(path + 'val.csv')
test = pd.read_csv(path + 'test.csv')
train

,Text,AL,MA,PPE,CDU,TP,OTH
0,To tell Or not to tell. Do you disclose to you...,0,0,0,0,0,1
1,Please need honest experience Please respond!!...,0,1,0,1,0,0
2,kratom and suboxone?. Is fhere a benefit to ta...,0,0,0,1,0,0
3,"Extraction. As I’m sure many of you know, subu...",0,1,0,1,0,0
4,Question on my prescription of suboxone?. So i...,0,1,0,0,0,0
...,...,...,...,...,...,...,...
4061,Down to 2mg a day. Finally successfully lowere...,0,0,0,1,1,0
4062,"Close To The Jump: What To Expect?. Hi all,\n\...",0,0,0,0,1,0
4063,Will smoking Salvia (Salvinorin A - high effic...,0,0,0,1,0,0
4064,Looking to get on Sub or Methadone Program - A...,1,0,0,0,0,0


In [ ]:
# Cleaning Data [Remove unncessary symbols]
def cleaning_data(row, bool_stopwords):
      row=str(row)
      post = row.replace('\n', ' ') #removing new line
      url_pattern = re.compile(r'https?://\S+|www\.\S+') # removing urls
      post = url_pattern.sub(r'', post)
      post = re.sub(r'[^\w\s|\.]', '', str(post)) #removing unnecessary punctuation #Not removing the full-stop
      post = post.lower() # Converting into lowercase
      if bool_stopwords == True:
        result = post.split()
        post = [word.strip() for word in result if word not in stp ]
        post =" ".join(post)
      return post

# Apply the function into the train dataset
# Stopwords are not removed
train['Text'] = train['Text'].apply(cleaning_data, bool_stopwords = False)
val['Text'] = val['Text'].apply(cleaning_data, bool_stopwords = False)
test['Text'] = test['Text'].apply(cleaning_data, bool_stopwords = False)
train

,Text,AL,MA,PPE,CDU,TP,OTH
0,to tell or not to tell. do you disclose to you...,0,0,0,0,0,1
1,please need honest experience please respond n...,0,1,0,1,0,0
2,kratom and suboxone. is fhere a benefit to tak...,0,0,0,1,0,0
3,extraction. as im sure many of you know subute...,0,1,0,1,0,0
4,question on my prescription of suboxone. so it...,0,1,0,0,0,0
...,...,...,...,...,...,...,...
4061,down to 2mg a day. finally successfully lowere...,0,0,0,1,1,0
4062,close to the jump what to expect. hi all i sa...,0,0,0,0,1,0
4063,will smoking salvia salvinorin a high efficac...,0,0,0,1,0,0
4064,looking to get on sub or methadone program au...,1,0,0,0,0,0


## Text Classification with KTrain

In [ ]:
!pip install ktrain

In [ ]:
!pip show ktrain


Name: ktrain
Version: 0.37.6
Summary: ktrain is a wrapper for TensorFlow Keras that makes deep learning and AI more accessible and easier to apply
Home-page: https://github.com/amaiya/ktrain
Author: Arun S. Maiya
Author-email: arun@maiya.net
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: cchardet, chardet, fastprogress, jieba, joblib, keras-bert, langdetect, matplotlib, packaging, pandas, requests, scikit-learn, sentencepiece, syntok, tika, transformers, whoosh
Required-by: 


In [ ]:
import ktrain
from ktrain import text

In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
from tkinter import X
target_cols = ['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']


In [ ]:
#### Multilabel
(X_train,y_train),(X_valid,y_valid), preprocess=text.texts_from_df(train_df = train,
                   text_column = 'Text',
                   label_columns = target_cols,
                   val_df = val,
                   ngram_range=2,
                   preprocess_mode='standard',
                   max_features = 50000,
                   maxlen = 256,
                   )

['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   0   0    0    0   0    1
1   0   1    0    1   0    0
2   0   0    0    1   0    0
3   0   1    0    1   0    0
4   0   1    0    0   0    0
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   1   1    0    0   0    0
1   1   0    0    0   0    0
2   0   0    0    1   1    0
3   0   0    0    0   1    0
4   0   1    0    0   0    0
language: en
Word Counts: 14509
Nrows: 4066
4066 train sequences
train sequence lengths:
	mean : 124
	95percentile : 284
	99percentile : 477
Adding 2-gram features
max_features changed to 200924 with addition of ngrams
Average train sequence length with ngrams: 247
train (w/ngrams) sequence lengths:
	mean : 247
	95percentile : 567
	99percentile : 954
x_train shape: (4066,256)
y_train shape: (4066, 6)
Is Multi-Label? True
508 test sequences
test sequence lengths:
	mean : 120
	95percentile : 276
	99percentile : 562
Average test sequence length with ngrams: 215
test (w

In [ ]:
#### For test set Processing
(X_train,y_train), (X_test,y_test),preprocess=text.texts_from_df(train_df = train,
                   text_column = 'Text',
                   label_columns = target_cols,
                   val_df = test,
                    ngram_range=2,
                   preprocess_mode='standard',
                   max_features = 50000,
                   maxlen = 256,
                  )

['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   0   0    0    0   0    1
1   0   1    0    1   0    0
2   0   0    0    1   0    0
3   0   1    0    1   0    0
4   0   1    0    0   0    0
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   0   1    1    0   0    0
1   0   0    1    0   1    0
2   0   0    0    0   0    1
3   0   0    1    0   1    0
4   0   0    0    0   0    1
language: en
Word Counts: 14509
Nrows: 4066
4066 train sequences
train sequence lengths:
	mean : 124
	95percentile : 284
	99percentile : 477
Adding 2-gram features
max_features changed to 200924 with addition of ngrams
Average train sequence length with ngrams: 247
train (w/ngrams) sequence lengths:
	mean : 247
	95percentile : 567
	99percentile : 954
x_train shape: (4066,256)
y_train shape: (4066, 6)
Is Multi-Label? True
509 test sequences
test sequence lengths:
	mean : 120
	95percentile : 281
	99percentile : 497
Average test sequence length with ngrams: 215
test (w

## Logistic Regression

In [ ]:
model = text.text_classifier('logreg', (X_train,y_train), preproc=preprocess)
learner = ktrain.get_learner(model, train_data=(X_train,y_train), val_data=(X_valid,y_valid))

Is Multi-Label? True
compiling word ID features...
maxlen is 256
done.


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 256, 64)           12859136  
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 256, 64)          0         
 lDropout1D)                                                     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
learner.autofit(0.0005, 10)



begin training using triangular learning rate policy with max lr of 0.0005...
Epoch 1/10
128/128 [==============================] - 2s 8ms/step - loss: 0.7092 - binary_accuracy: 0.6177 - val_loss: 0.6273 - val_binary_accuracy: 0.7402
Epoch 2/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5957 - binary_accuracy: 0.7054 - val_loss: 0.6207 - val_binary_accuracy: 0.7402
Epoch 3/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5688 - binary_accuracy: 0.7223 - val_loss: 0.6053 - val_binary_accuracy: 0.7398
Epoch 4/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5485 - binary_accuracy: 0.7314 - val_loss: 0.5782 - val_binary_accuracy: 0.7441
Epoch 5/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5345 - binary_accuracy: 0.7391 - val_loss: 0.5450 - val_binary_accuracy: 0.7562
Epoch 6/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5213 - binary_accuracy: 0.7468 - val_loss: 0.5205 - val_binary_a

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
# save logreg model
path= '/content/drive/MyDrive/Omar/AAAI-Submission/ML-Models/'
predictor.save(path+'lr')

In [ ]:
#load model
path= '/content/drive/MyDrive/Omar/AAAI-Submission/ML-Models/'
predictor = ktrain.load_predictor(path+'lr')

In [ ]:
predictor.predict(test['Text'][0])

[('AL', 0.21064168),
 ('MA', 0.45055705),
 ('PPE', 0.4258285),
 ('CDU', 0.40839165),
 ('TP', 0.33680677),
 ('OTH', 0.1751357)]

In [ ]:
y_pred = predictor.model.predict(X_test, verbose=0)
new_pred = y_pred > 0.4
new_labels = new_pred.astype('uint8')
print(new_labels)

[[0 1 1 1 0 0]
 [0 0 1 0 1 0]
 [1 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 1 0 0]]


In [ ]:
def saving_predictions(lst, mdl_nm):
  labelss = ['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
  xx = []
  for ls in lst:
      xy = []
      for i in range(len(ls)):
          if(ls[i]==1):
            xy.append(labelss[i])
      xx.append(xy)
  print(len(xx), xx)
  df = pd.DataFrame({'Predictions': xx})
  pathx = '/content/drive/MyDrive/Omar/AAAI-Submission/ChatGPT-Exps/'
  df.to_csv(pathx+f'{mdl_nm}-Predictions.csv', index= False)


In [ ]:
saving_predictions(new_labels, 'LR')

509 [['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['AL'], ['MA'], [], ['PPE'], ['PPE', 'TP'], [], ['AL'], ['MA'], ['MA'], [], ['PPE', 'TP'], ['MA', 'PPE'], ['MA'], ['MA', 'PPE', 'CDU'], ['AL'], ['MA'], ['MA'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], ['MA', 'PPE'], ['MA'], [], [], [], ['AL'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], [], [], [], ['MA', 'PPE'], ['AL'], [], ['AL'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE', 'CDU'], [], [], ['MA', 'PPE'], [], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['MA'], ['PPE'], ['AL'], ['MA', 'PPE'], ['MA', 'PPE', 'CDU'], ['PPE', 'CDU', 'TP'], ['MA', 'CDU'], [], ['PPE', 'TP'], ['MA'], ['AL'], ['AL'], ['MA', 'PPE'], ['AL'], ['AL'], ['PPE'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['MA', 'CDU'], [], ['MA', 'PPE', 'CDU', 'TP'], ['AL'], ['MA', 'PPE', 'CDU'], ['MA', 'PPE', 'CDU'], ['AL'], ['MA', 'CDU'], [], ['MA', 'PPE', 'CDU'], [], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], [], ['MA', 'PPE', 'TP'], [], ['AL'], [], ['MA'], ['MA', 'P

In [ ]:
from sklearn.metrics import classification_report

#Show precision and recall per genre
print(classification_report(y_test, new_labels,target_names=target_cols))
print("F1-Score:",f1_score(y_test,new_labels,average='weighted'))
print("Precision:",precision_score(y_test,new_labels,average='weighted'))
print("Recall:",recall_score(y_test,new_labels,average='weighted'))



              precision    recall  f1-score   support

          AL       0.63      0.68      0.65        77
          MA       0.61      0.64      0.63       180
         PPE       0.49      0.66      0.56       174
         CDU       0.71      0.57      0.63       160
          TP       0.74      0.64      0.68       140
         OTH       1.00      0.02      0.04        45

   micro avg       0.61      0.60      0.61       776
   macro avg       0.70      0.53      0.53       776
weighted avg       0.65      0.60      0.59       776
 samples avg       0.51      0.58      0.51       776

F1-Score: 0.5930820447546286
Precision: 0.6531187615733789
Recall: 0.5979381443298969


##NBSVM

In [ ]:
model = text.text_classifier('nbsvm', (X_train,y_train), preproc=preprocess)
learner = ktrain.get_learner(model, train_data=(X_train,y_train), val_data=(X_valid,y_valid))

Is Multi-Label? True
compiling word ID features...
maxlen is 256
done.


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 256, 64)           12859136  
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 256, 64)          0         
 lDropout1D)                                                     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 batch_normalization_2 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
learner.autofit(0.0005, 10)



begin training using triangular learning rate policy with max lr of 0.0005...
Epoch 1/10
128/128 [==============================] - 2s 8ms/step - loss: 0.7035 - binary_accuracy: 0.6206 - val_loss: 0.6283 - val_binary_accuracy: 0.7375
Epoch 2/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5871 - binary_accuracy: 0.7094 - val_loss: 0.6203 - val_binary_accuracy: 0.7398
Epoch 3/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5611 - binary_accuracy: 0.7279 - val_loss: 0.6023 - val_binary_accuracy: 0.7454
Epoch 4/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5428 - binary_accuracy: 0.7361 - val_loss: 0.5657 - val_binary_accuracy: 0.7644
Epoch 5/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5247 - binary_accuracy: 0.7458 - val_loss: 0.5244 - val_binary_accuracy: 0.7753
Epoch 6/10
128/128 [==============================] - 1s 7ms/step - loss: 0.5092 - binary_accuracy: 0.7536 - val_loss: 0.4984 - val_binary_a

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
# save logreg model
predictor.save(path+'nbsvm')

In [ ]:
#load model
predictor = ktrain.load_predictor(path+'nbsvm')

In [ ]:
y_pred = predictor.model.predict(X_test, verbose=0)
new_pred = y_pred > 0.4
new_labels = new_pred.astype('uint8')

In [ ]:
saving_predictions(new_labels, 'NBSVM')

509 [['PPE'], ['PPE', 'TP'], [], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE'], ['PPE', 'CDU', 'TP'], ['AL'], ['AL'], ['MA', 'PPE', 'CDU'], ['MA', 'CDU'], ['MA'], ['PPE', 'TP'], ['PPE', 'CDU', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['PPE', 'TP'], ['MA', 'CDU'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['PPE', 'TP'], ['MA', 'CDU'], [], ['MA'], ['AL'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], [], [], [], ['PPE', 'TP'], ['MA'], ['MA', 'PPE'], ['AL', 'PPE'], ['AL', 'MA', 'CDU'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE', 'CDU'], ['CDU'], ['MA'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], ['MA', 'PPE'], ['MA', 'CDU'], ['TP'], ['AL'], ['AL'], ['PPE', 'TP'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE'], ['MA', 'PPE', 'TP'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'CDU', 'TP'], ['MA', 'PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['AL'], [], ['PPE'], ['MA'], ['AL'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE'], ['MA', 'CDU'], ['MA', 'PPE', 'TP'], ['MA', 'PPE'], ['MA', 'PPE', 'CDU

In [ ]:
from sklearn.metrics import classification_report

#Show precision and recall per genre
print(classification_report(y_test, new_labels,target_names=target_cols))
print("F1-Score:",f1_score(y_test,new_labels,average='weighted'))
print("Precision:",precision_score(y_test,new_labels,average='weighted'))
print("Recall:",recall_score(y_test,new_labels,average='weighted'))

              precision    recall  f1-score   support

          AL       0.73      0.57      0.64        77
          MA       0.58      0.73      0.65       180
         PPE       0.44      0.75      0.56       174
         CDU       0.61      0.62      0.62       160
          TP       0.72      0.78      0.75       140
         OTH       0.50      0.02      0.04        45

   micro avg       0.58      0.66      0.62       776
   macro avg       0.60      0.58      0.54       776
weighted avg       0.59      0.66      0.60       776
 samples avg       0.52      0.62      0.54       776

F1-Score: 0.6029703485522578
Precision: 0.5924585848765298
Recall: 0.6623711340206185


## BiGRU

In [ ]:
#### Multilabel
### DATA must be processed with ngram_range=1 for BiGRU model
(X_train,y_train),(X_valid,y_valid), preprocess=text.texts_from_df(train_df = train,
                   text_column = 'Text',
                   label_columns = target_cols,
                   val_df = val,
                   ngram_range=1,
                   preprocess_mode='standard',
                   max_features = 50000,
                   maxlen = 256,
                   )
#### For test set Processing
(X_train,y_train), (X_test,y_test),preprocess=text.texts_from_df(train_df = train,
                   text_column = 'Text',
                   label_columns = target_cols,
                   val_df = test,
                    ngram_range=1,
                   preprocess_mode='standard',
                   max_features = 50000,
                   maxlen = 256,
                  )

['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   0   0    0    0   0    1
1   0   1    0    1   0    0
2   0   0    0    1   0    0
3   0   1    0    1   0    0
4   0   1    0    0   0    0
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   1   1    0    0   0    0
1   1   0    0    0   0    0
2   0   0    0    1   1    0
3   0   0    0    0   1    0
4   0   1    0    0   0    0
language: en
Word Counts: 14509
Nrows: 4066
4066 train sequences
train sequence lengths:
	mean : 124
	95percentile : 284
	99percentile : 477
x_train shape: (4066,256)
y_train shape: (4066, 6)
Is Multi-Label? True
508 test sequences
test sequence lengths:
	mean : 120
	95percentile : 276
	99percentile : 562
x_test shape: (508,256)
y_test shape: (508, 6)
['AL', 'MA', 'PPE', 'CDU', 'TP', 'OTH']
   AL  MA  PPE  CDU  TP  OTH
0   0   0    0    0   0    1
1   0   1    0    1   0    0
2   0   0    0    1   0    0
3   0   1    0    1   0    0
4   0   1    0    0   0    0
['AL

In [ ]:
model = text.text_classifier('bigru', (X_train,y_train), preproc=preprocess)
learner = ktrain.get_learner(model, train_data=(X_train,y_train), val_data=(X_valid,y_valid))

Is Multi-Label? True
compiling word ID features...
maxlen is 256
word vectors will be loaded from: https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
processing pretrained word vectors...
loading pretrained word vectors...this may take a few moments...


done.


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 256, 300)     15000000    ['input_2[0][0]']                
                                                                                                  
 spatial_dropout1d_4 (SpatialDr  (None, 256, 300)    0           ['embedding_4[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 256, 160)    183360      ['spatial_dropout1d_4[0][0]

In [ ]:
learner.autofit(0.0005, 10)



begin training using triangular learning rate policy with max lr of 0.0005...
Epoch 1/10
128/128 [==============================] - 7s 29ms/step - loss: 0.5271 - binary_accuracy: 0.7377 - val_loss: 0.4726 - val_binary_accuracy: 0.7641
Epoch 2/10
128/128 [==============================] - 3s 25ms/step - loss: 0.4251 - binary_accuracy: 0.7952 - val_loss: 0.3976 - val_binary_accuracy: 0.8228
Epoch 3/10
128/128 [==============================] - 3s 23ms/step - loss: 0.3625 - binary_accuracy: 0.8361 - val_loss: 0.3593 - val_binary_accuracy: 0.8363
Epoch 4/10
128/128 [==============================] - 3s 25ms/step - loss: 0.3190 - binary_accuracy: 0.8606 - val_loss: 0.3434 - val_binary_accuracy: 0.8461
Epoch 5/10
128/128 [==============================] - 3s 23ms/step - loss: 0.2886 - binary_accuracy: 0.8772 - val_loss: 0.3290 - val_binary_accuracy: 0.8497
Epoch 6/10
128/128 [==============================] - 3s 24ms/step - loss: 0.2611 - binary_accuracy: 0.8911 - val_loss: 0.3242 - val_bi

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)
# save logreg model
predictor.save(path+'bigru')

In [ ]:
#load model
predictor = ktrain.load_predictor(path+'bigru')

In [ ]:
y_pred = predictor.model.predict(X_test, verbose=0)
new_pred = y_pred > 0.45
new_labels = new_pred.astype('uint8')

In [ ]:
saving_predictions(new_labels, 'BiGRU')

509 [['MA'], ['PPE', 'TP'], ['OTH'], ['MA', 'PPE'], [], ['PPE', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], ['AL', 'OTH'], ['AL', 'OTH'], ['CDU'], [], ['PPE'], ['TP'], ['MA', 'PPE'], ['MA'], ['CDU', 'TP'], ['OTH'], ['MA', 'PPE'], ['MA'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['TP'], ['MA', 'CDU'], [], ['PPE', 'OTH'], ['AL'], ['TP'], ['MA', 'CDU'], ['OTH'], ['MA'], ['OTH'], ['PPE', 'TP'], [], ['AL'], ['AL'], ['AL'], ['PPE', 'TP'], ['OTH'], ['MA', 'CDU'], ['CDU'], ['OTH'], ['CDU'], ['PPE', 'TP'], ['PPE'], [], ['TP'], ['AL'], ['MA'], ['TP'], ['PPE', 'CDU', 'TP'], ['AL'], ['MA'], ['MA', 'CDU'], ['TP'], ['MA', 'CDU'], ['CDU', 'TP'], ['PPE', 'TP'], ['PPE', 'CDU'], ['AL'], ['AL'], ['PPE'], ['AL'], [], ['PPE', 'CDU', 'TP'], ['MA', 'CDU'], ['PPE', 'TP'], ['MA', 'CDU'], ['MA', 'PPE', 'TP'], [], [], ['MA', 'CDU'], ['MA'], [], ['MA'], [], ['MA', 'PPE'], ['PPE', 'TP'], ['MA', 'CDU'], ['PPE', 'TP'], ['TP'], ['MA', 'PPE'], ['CDU'], ['AL'], [], ['MA'], [], ['OTH'], ['AL'], ['OTH'], ['CDU'], ['AL'

In [ ]:
from sklearn.metrics import classification_report

#Show precision and recall per genre
print(classification_report(y_test, new_labels,target_names=target_cols))
print("F1-Score:",f1_score(y_test,new_labels,average='weighted'))
print("Precision:",precision_score(y_test,new_labels,average='weighted'))
print("Recall:",recall_score(y_test,new_labels,average='weighted'))

              precision    recall  f1-score   support

          AL       0.72      0.69      0.70        77
          MA       0.73      0.68      0.70       180
         PPE       0.61      0.61      0.61       174
         CDU       0.80      0.72      0.76       160
          TP       0.80      0.84      0.82       140
         OTH       0.51      0.51      0.51        45

   micro avg       0.71      0.69      0.70       776
   macro avg       0.69      0.67      0.68       776
weighted avg       0.72      0.69      0.70       776
 samples avg       0.69      0.69      0.66       776

F1-Score: 0.7025307201887028
Precision: 0.7159478752910313
Recall: 0.6907216494845361


## Fasttext model

In [ ]:
model = text.text_classifier('fasttext', (X_train,y_train), preproc=preprocess)
learner = ktrain.get_learner(model, train_data=(X_train,y_train), val_data=(X_valid,y_valid))

Is Multi-Label? True
compiling word ID features...
maxlen is 256
done.


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 256, 64)           3200000   
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 256, 64)          0         
 lDropout1D)                                                     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 batch_normalization_3 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
learner.autofit(0.0005, 10)



begin training using triangular learning rate policy with max lr of 0.0005...
Epoch 1/10
128/128 [==============================] - 2s 7ms/step - loss: 0.7131 - binary_accuracy: 0.6147 - val_loss: 0.6192 - val_binary_accuracy: 0.7402
Epoch 2/10
128/128 [==============================] - 1s 6ms/step - loss: 0.5788 - binary_accuracy: 0.7216 - val_loss: 0.6054 - val_binary_accuracy: 0.7474
Epoch 3/10
128/128 [==============================] - 1s 6ms/step - loss: 0.5496 - binary_accuracy: 0.7359 - val_loss: 0.5788 - val_binary_accuracy: 0.7628
Epoch 4/10
128/128 [==============================] - 1s 6ms/step - loss: 0.5335 - binary_accuracy: 0.7430 - val_loss: 0.5375 - val_binary_accuracy: 0.7710
Epoch 5/10
128/128 [==============================] - 1s 6ms/step - loss: 0.5124 - binary_accuracy: 0.7557 - val_loss: 0.4950 - val_binary_accuracy: 0.7854
Epoch 6/10
128/128 [==============================] - 1s 6ms/step - loss: 0.4970 - binary_accuracy: 0.7642 - val_loss: 0.4691 - val_binary_a

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)
# save logreg model
predictor.save(path+'fasttext')

In [ ]:
#load model
predictor = ktrain.load_predictor(path+'fasttext')

In [ ]:
y_pred = predictor.model.predict(X_test, verbose=0)
new_pred = y_pred > 0.4
new_labels = new_pred.astype('uint8')

In [ ]:
saving_predictions(new_labels, 'FastText')

509 [['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE', 'TP'], [], ['PPE'], ['PPE', 'TP'], ['AL'], ['AL'], ['MA', 'PPE', 'CDU'], ['MA'], [], ['PPE', 'TP'], ['MA', 'PPE'], ['MA'], ['PPE', 'CDU', 'TP'], ['AL'], ['MA'], ['MA'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'CDU', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], [], [], [], ['AL'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], [], [], ['PPE', 'CDU', 'TP'], ['MA', 'PPE'], ['AL'], ['AL'], ['AL'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE', 'CDU'], [], [], ['MA', 'PPE', 'CDU', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['AL'], ['MA', 'CDU'], ['PPE', 'TP'], ['PPE', 'TP'], ['AL'], ['MA', 'PPE'], ['MA', 'PPE', 'CDU', 'TP'], ['PPE', 'CDU', 'TP'], ['MA', 'CDU'], ['PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['PPE'], ['AL'], ['AL'], ['MA', 'PPE'], ['AL'], ['PPE'], ['MA', 'PPE', 'CDU', 'TP'], ['MA', 'PPE', 'CDU'], ['PPE', 'TP'], ['MA', 'CDU'], ['PPE', 'TP'], ['MA', 'PPE', 'CDU', 'TP'], ['PPE', 'TP'], ['MA'

In [ ]:
from sklearn.metrics import classification_report

#Show precision and recall per genre
print(classification_report(y_test, new_labels,target_names=target_cols))
print("F1-Score:",f1_score(y_test,new_labels,average='weighted'))
print("Precision:",precision_score(y_test,new_labels,average='weighted'))
print("Recall:",recall_score(y_test,new_labels,average='weighted'))

              precision    recall  f1-score   support

          AL       0.64      0.73      0.68        77
          MA       0.61      0.70      0.65       180
         PPE       0.50      0.78      0.61       174
         CDU       0.63      0.64      0.63       160
          TP       0.69      0.84      0.75       140
         OTH       1.00      0.02      0.04        45

   micro avg       0.60      0.69      0.64       776
   macro avg       0.68      0.62      0.56       776
weighted avg       0.63      0.69      0.62       776
 samples avg       0.55      0.65      0.57       776

F1-Score: 0.6247890425669224
Precision: 0.6287694710765653
Recall: 0.6932989690721649
